In [1]:
from importlib.machinery import SourceFileLoader
import pandas as pd
import numpy  as np

wr = SourceFileLoader("mobius", r"../../mobius.py").load_module()
cu = SourceFileLoader("mobius_calib_uncert_lmfit", r"../../mobius_calib_uncert_lmfit.py").load_module()

multi = SourceFileLoader("multiloop", r"../multiloop.py").load_module()
magicopt = SourceFileLoader("magicopt",  r"../magicopt.py").load_module()

In [2]:
wr.initialize('../../../Applications/MAGIC/magic_simple.dll')

parfile = 'templateparameters.dat'
infile  = 'templateinputs.dat'

dataset = wr.DataSet.setup_from_parameter_and_input_files(parfile, infile)

In [3]:
idx = ('Soil', 'Lake')

In [4]:
def do_optim(ds, ID, soil, depo, lake, printout=True) :

    #ds.set_parameter_double('Initial exchangeable Ca on soil as % of CEC', ['Soil'], soil['ExCa'])
    #ds.set_parameter_double('Initial exchangeable Mg on soil as % of CEC', ['Soil'], soil['ExMg'])
    #ds.set_parameter_double('Initial exchangeable Na on soil as % of CEC', ['Soil'], soil['ExNa'])
    #ds.set_parameter_double('Initial exchangeable K on soil as % of CEC', ['Soil'], soil['ExK'])
    
    params_a, comparisons_a = magicopt.get_acid_anion_setup(ds, idx, 'Observed')
    
    res_pars1, nfev1 = magicopt.calib(ds, params_a, comparisons_a, method='nelder')# method='leastsq')
    
    #ds.set_parameter_double('Initial exchangeable Ca on soil as % of CEC', ['Soil'], soil['ECa-0'])
    #ds.set_parameter_double('Initial exchangeable Mg on soil as % of CEC', ['Soil'], soil['EMg-0'])
    #ds.set_parameter_double('Initial exchangeable Na on soil as % of CEC', ['Soil'], soil['ENa-0'])
    #ds.set_parameter_double('Initial exchangeable K on soil as % of CEC', ['Soil'], soil['EK-0'])
    
    params, comparisons = magicopt.get_base_cations_setup(ds, 
                            idx, 'Observed', usemax=False, obsE=True)
    
    #params.pretty_print()
    #Set initial guess to the present-day value
    params['ECa_S'].value = soil['ExCa']+20
    params['EMg_S'].value = soil['ExMg']
    params['ENa_S'].value = soil['ExNa']
    params['EK_S'].value = soil['ExK']
    
    params['ECa_S'].max = min(90.0, soil['ExCa']*4.0+15.0)
    params['EMg_S'].max = min(50.0, soil['ExMg']*4.0+10.0)
    params['ENa_S'].max = min(25.0, soil['ExNa']*2.0+5.0)
    params['EK_S'].max  = min(15.0, soil['ExK']*2.0+5.0)
    
    params['WCa_S'].max = soil['WeCa']*1.5+150.0
    params['WMg_S'].max = soil['WeMg']*1.2+50.0
    params['WNa_S'].max = soil['WeNa']*0.6+50.0
    params['WK_S'].max  = soil['WeK']*1.2+25.0
    
    #params['WCa_S'].value*=0.2
    #params['WMg_S'].value*=0.2
    #params['WNa_S'].value*=0.2
    #params['WK_S'].value*=0.2
    
    params['WK_S'].min = max(0.0, soil['K'] - depo['K'])    # soil['K'] is the sink
    
    
    ntries = 0
    err=0
    abserr=0
    
    #skip = True
    
    while (abserr>1 and ntries < 10) or ntries==0 :
        if ntries > 0 : print('retry %d'%ntries)
        if err < 0. :
            params['WCa_S'].value*=2
            if ID==3096:
                params['ECa_S'].value+=5
        elif err > 0.:
            params['WCa_S'].value/=2
            if ID==3096:
                params['ECa_S'].value-=5

        res_pars2, nfev = magicopt.calib(ds, params, comparisons, 
                                    #method='nelder'
                                    method='leastsq'
                                    )

        sim = ds.get_result_series('Ca(2+) ionic concentration', ['Lake'])
        obs = ds.get_input_series('Observed Ca', [], alignwithresults=True)
        err = np.nansum(sim-obs)
        abserr = np.nansum(np.abs(sim-obs))

        ntries += 1
        #if skip : break
    
    #if np.abs(err)>1.:
    #    high_error_ids.append((ID,err))
    #    print('High Ca error (abs>1)')

    if printout :
        comp = comparisons
        comp += comparisons_a
        comp += [('SO4(2-) ionic concentration', ['Lake'], 'Observed SO4', [])]

        for comparison in comp :
            simname, simindexes, obsname, obsindexes, *_ = comparison

            sim = ds.get_result_series(simname, simindexes)
            obs = ds.get_input_series(obsname, obsindexes, alignwithresults=True)

            err = np.sqrt(np.nansum(np.square(sim-obs)))

            print('Error of %s: RSSE(sim-obs): %f' % (obsname, err))

        print('\n')
        res_pars1.pretty_print()
        res_pars2.pretty_print()
        print('\n')
        
        ds.write_parameters_to_file('MobiusFiles/params_%d.dat'%ID)
        ds.write_inputs_to_file('MobiusFiles/inputs_%d.dat'%ID)
    
    return res_pars1, res_pars2

In [6]:
#soilfile, lakefile1, lakefile2, depofile, seqfile = excelfiles
#excelfiles = ('CCE-soil BJC.xls', 'CCE-lake BJC.xls', 'WRI-lake 2019.xls', 'CCE-depo BJC.xls', 'CCE-seqs BJC.xls')

excelfiles = ('Final_xlsx/Tusen1995-soil-2022.xlsx',
             'Final_xlsx/Tusen1995-lake-2022.xlsx',
             'Final_xlsx/Tusen2019-lake-2022.xlsx',
             'Final_xlsx/Tusen1995-depo-2022.xlsx',
             'Final_xlsx/Tusen1995-hseqs-2022.xlsx')

#multi.do_magic_loop(dataset, excelfile, do_fuzzy, do_id=9)
multi.do_magic_loop(dataset, excelfiles, do_optim, two_year=False, do_id=9)#, limit_num = 10)


#print('IDs with error in Ca > 1: ', high_error_ids)

Running lake Hokksjøen (ID 9)

retry 1
retry 2
retry 3
retry 4
retry 5
retry 6
retry 7
retry 8
retry 9
Error of Observed Ca: RSSE(sim-obs): 2.936933
Error of Observed Mg: RSSE(sim-obs): 1.631076
Error of Observed Na: RSSE(sim-obs): 0.165424
Error of Observed K: RSSE(sim-obs): 0.219894
Error of Observed ECa: RSSE(sim-obs): 0.719181
Error of Observed EMg: RSSE(sim-obs): 0.399693
Error of Observed ENa: RSSE(sim-obs): 0.117762
Error of Observed EK: RSSE(sim-obs): 0.221544
Error of Observed NO3: RSSE(sim-obs): 6.039778
Error of Observed SO4: RSSE(sim-obs): 0.066284


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     58.45     0.01    88.54    3.536     True     None     None
EK_S      9.465     0.01       15   0.8208     True     None     None
EMg_S      15.9     0.01    36.41    2.258     True     None     None